In [10]:
import pandas as pd 
from datetime import datetime
from dateutil.relativedelta import relativedelta
current_date = datetime.now()
one_month_earlier = current_date - relativedelta(months=1)
date = one_month_earlier.strftime('%Y-%m-%d')
from boxoffice_api import BoxOffice 
box_office = BoxOffice(api_key="bc442c47")
box_office = BoxOffice(outputformat="DF")
x= box_office.get_quarterly(1,2024)
x


Rank                  Release        Gross Theaters   Total Gross   
0      1                    Wonka  $81,895,255    4,213  $215,025,157  \
1      2     Bob Marley: One Love  $75,341,080    3,597   $75,341,080   
2      3               Mean Girls  $72,245,214    3,826   $72,245,214   
3      4                Migration  $66,852,925    3,839  $120,956,880   
4      5            The Beekeeper  $63,814,446    3,337   $63,814,446   
..   ...                      ...          ...      ...           ...   
145  146                  Fremont       $1,643       32      $271,343   
146  147       My Sailor, My Love         $851       37       $88,935   
147  148              After Death         $705    2,730   $11,478,326   
148  149                King Coal         $370        -       $54,051   
149  150  Day by Day: The Dynasty         $292        6       $12,929   

    Release Date                Distributor  
0         Dec 15               Warner Bros.  
1         Feb 14         Paramount Pictures  
2         Jan 12         Paramount Pictures  
3         Dec 22         Universal Pictures  
4         Jan 12  Metro-Goldwyn-Mayer (MGM)  
..           ...                        ...  
145       Aug 25            Music Box Films  
146       Sep 22            Music Box Films  
147       Oct 27              Angel Studios  
148       Aug 11            Fourth Act Film  
149        Dec 8                     Utopia  

[150 rows x 7 columns]

In [32]:
from fastapi import FastAPI, HTTPException
import requests

def get_batman_movies():
    api_key = "bc442c47"  
    movie_id = "tt3896198"   
    url = f"http://www.omdbapi.com/?i={movie_id}&apikey={api_key}"
    
    response = requests.get(url)
    if response.status_code != 200:
        return "Error fetching movie details", {}
    movies = response.json()
    movies_images_url = f"http://img.omdbapi.com/?i={movie_id}&apikey={api_key}"

    return movies, {"image_url": movies_images_url}

x = get_batman_movies()
print(x)




({'Title': 'Guardians of the Galaxy Vol. 2', 'Year': '2017', 'Rated': 'PG-13', 'Released': '05 May 2017', 'Runtime': '136 min', 'Genre': 'Action, Adventure, Comedy', 'Director': 'James Gunn', 'Writer': 'James Gunn, Dan Abnett, Andy Lanning', 'Actors': 'Chris Pratt, Zoe Saldana, Dave Bautista', 'Plot': "The Guardians struggle to keep together as a team while dealing with their personal family issues, notably Star-Lord's encounter with his father, the ambitious celestial being Ego.", 'Language': 'English', 'Country': 'United States', 'Awards': 'Nominated for 1 Oscar. 15 wins & 60 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNjM0NTc0NzItM2FlYS00YzEwLWE0YmUtNTA2ZWIzODc2OTgxXkEyXkFqcGdeQXVyNTgwNzIyNzg@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '85%'}, {'Source': 'Metacritic', 'Value': '67/100'}], 'Metascore': '67', 'imdbRating': '7.6', 'imdbVotes': '750,086', 'imdbID': 'tt3896198', '

In [26]:
from fastapi import FastAPI, Request
from fastapi.templating import Jinja2Templates
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from boxoffice_api import BoxOffice  # Ensure this is correctly imported based on your actual setup
import os


OMDB_API_KEY = "bc442c47"
year= "2024"
import requests

def get_movie_details(title, OMDB_API_KEY):
    """Fetch movie details including the poster from OMDB API by title."""
    # Correctly format the URL with the query parameters
    url = f"http://www.omdbapi.com/?t={title}&apikey={OMDB_API_KEY}&y={year}&plot=full"
    response = requests.get(url)
    movie = response.json()
    if movie["Response"] == "True":
        return movie
    else:
        return {
            "title": title,
            "poster": None,
            "box_office": "N/A",
            "release_date": "N/A"
        }

In [27]:
get_movie_details("Dune" ,OMDB_API_KEY )

{'Title': 'Dune: Part Two',
 'Year': '2024',
 'Rated': 'PG-13',
 'Released': '01 Mar 2024',
 'Runtime': '166 min',
 'Genre': 'Action, Adventure, Drama',
 'Director': 'Denis Villeneuve',
 'Writer': 'Denis Villeneuve, Jon Spaihts, Frank Herbert',
 'Actors': 'Florence Pugh, Timothée Chalamet, Rebecca Ferguson',
 'Plot': "Duke Paul Atreides joins the Fremen and begins a spiritual and martial journey to become Muad'dib, while trying to prevent the horrible but inevitable future he's witnessed: a Holy War in his name, spreading throughout the known universe.",
 'Language': 'English',
 'Country': 'United States, Canada',
 'Awards': '2 nominations',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BODI0YjNhNjUtYjM0My00MTUwLWFlYTMtMWI2NGUzYjNjNGQzXkEyXkFqcGdeQXVyMDM2NDM2MQ@@._V1_SX300.jpg',
 'Ratings': [],
 'Metascore': 'N/A',
 'imdbRating': 'N/A',
 'imdbVotes': 'N/A',
 'imdbID': 'tt15239678',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': 'N/A',
 'Production': 'N/A',
 'Website': 'N/A',
 'Res

In [41]:
def read_root(request: Request):
    date = datetime.now()
    one_month_earlier = current_date - relativedelta(months=1)
    date = one_month_earlier.strftime('%Y-%m-%d')
    
    # Assuming BoxOffice API setup is similar and correct
    box_office = BoxOffice(api_key="bc442c47")
    box_office_data = box_office.get_daily(date)
    movies_df = pd.DataFrame(box_office_data)
    
    # Fetch movie details from OMDB for each movie
    movies_details = [get_movie_details(title , OMDB_API_KEY) for title in movies_df['Title']]
    movies_df = pd.DataFrame(movies_details)
    
    return movies_df

In [51]:
date = datetime.now()
one_month_earlier = current_date - relativedelta(day=2)
date = one_month_earlier.strftime('%Y-%m-%d')
box_office_data = box_office.get_daily(date)
print(box_office_data)

    TD  YD                            Release       Daily    %± YD   %± LW   
0    1   -                            Argylle  $6,485,575        -       -  \
1    2   1        The Chosen: S4 Episodes 1-3  $1,760,021   +22.7%       -   
2    3   2                      The Beekeeper  $1,306,392    +143%  -27.7%   
3    4   3                     Anyone But You  $1,055,693    +162%  -29.5%   
4    5   6                              Wonka  $1,034,210  +241.2%  -23.3%   
5    6   4                         Mean Girls  $1,017,228  +156.8%  -46.8%   
6    7   9                          Migration    $847,510  +274.6%  -18.6%   
7    8   8                   American Fiction    $616,306  +129.2%  -21.1%   
8    9   5                        Poor Things    $593,339   +83.5%  -31.2%   
9   10  11       Aquaman and the Lost Kingdom    $442,636    +185%  -33.1%   
10  11  10               The Boys in the Boat    $381,967   +83.2%  -36.8%   
11  12  13                         Night Swim    $375,695  +213.

In [14]:
box_office = BoxOffice(api_key="bc442c47")
box_office = BoxOffice(outputformat="DF")
box_office_data= box_office.get_quarterly(1,2024)
  
movies_df = pd.DataFrame(box_office_data)
movies_df = movies_df[movies_df['Release'] != 'Mean Girls']
movies_df

Rank                  Release        Gross Theaters   Total Gross   
0      1                    Wonka  $78,631,868    4,213  $211,761,770  \
2      3                Migration  $63,065,270    3,839  $117,169,225   
3      4            The Beekeeper  $60,965,947    3,337   $60,965,947   
4      5           Anyone But You  $60,753,508    3,055   $85,590,893   
5      6     Bob Marley: One Love  $56,088,180    3,597   $56,088,180   
..   ...                      ...          ...      ...           ...   
124  125      The Unknown Country       $1,945       10       $69,172   
125  126                  Fremont       $1,615       32      $271,315   
126  127              After Death         $705    2,730   $11,478,326   
127  128                King Coal         $370        -       $54,051   
128  129  Day by Day: The Dynasty         $292        6       $12,929   

    Release Date                Distributor  
0         Dec 15               Warner Bros.  
2         Dec 22         Universal Pictures  
3         Jan 12  Metro-Goldwyn-Mayer (MGM)  
4         Dec 22          Columbia Pictures  
5         Feb 14         Paramount Pictures  
..           ...                        ...  
124       Jul 28            Music Box Films  
125       Aug 25            Music Box Films  
126       Oct 27              Angel Studios  
127       Aug 11            Fourth Act Film  
128        Dec 8                     Utopia  

[128 rows x 7 columns]

In [10]:
import requests
import pandas as pd

url = "https://movies-api14.p.rapidapi.com/shows"

headers = {
	"X-RapidAPI-Key": "341580146bmsh6d7bf77c4ad811ep1d8340jsn825f96cd7f0a",
	"X-RapidAPI-Host": "movies-api14.p.rapidapi.com"
}

def fetch_page(page_no):
    params = {'page': page_no}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Assuming the API returns a list of items per page
all_data = []  # Initialize an empty list to store all records

page = 1
max_pages = 5  # Assuming you want to fetch up to 5 pages; adjust as needed

while page <= max_pages:
    data = fetch_page(page)

    
    page += 1  # Move to the next page

# Convert the collected data to a DataFrame
data


{'movies': [{'_id': 66840,
   'backdrop_path': 'https://image.tmdb.org/t/p/original/m731O4mEUT6Z9iiFvwBAFOE2ejp.jpg',
   'first_aired': '2016-09-20',
   'genres': ['Crime', 'Drama'],
   'original_title': 'Bull',
   'overview': 'Dr. Jason Bull is the brilliant, brash, and charming founder of a hugely successful trial consulting firm.',
   'poster_path': 'https://image.tmdb.org/t/p/original/r6bM2rNjSk5b9gWBOOKCVgKbjc0.jpg',
   'title': 'Bull',
   'contentType': 'show'},
  {'_id': 60622,
   'backdrop_path': 'https://image.tmdb.org/t/p/original/8xud7NgLcw4BXD9LnqixbfMgciG.jpg',
   'first_aired': '2014-04-15',
   'genres': ['Crime', 'Drama'],
   'original_title': 'Fargo',
   'overview': 'A close-knit anthology series dealing with stories involving malice, violence and murder based in and around Minnesota.',
   'poster_path': 'https://image.tmdb.org/t/p/original/9ZIhl17uFlXCNUputSEDHwZYIEJ.jpg',
   'title': 'Fargo',
   'contentType': 'show'},
  {'_id': 1413,
   'backdrop_path': 'https://imag

In [41]:
import requests

url = "https://ott-details.p.rapidapi.com/advancedsearch"

querystring = {"start_year":"2023","end_year":"2023","type":"movie","sort":"latest","page":"1"}

headers = {
	"X-RapidAPI-Key": "341580146bmsh6d7bf77c4ad811ep1d8340jsn825f96cd7f0a",
	"X-RapidAPI-Host": "ott-details.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

response.json()

{'page': 1, 'results': []}

In [44]:
import requests
title = "Dune"

url = "https://moviesdatabase.p.rapidapi.com/titles/{title}"

headers = {
	"X-RapidAPI-Key": "341580146bmsh6d7bf77c4ad811ep1d8340jsn825f96cd7f0a",
	"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
response.json()

{'results': None}

In [51]:
import requests

url = "https://imdb-top-100-movies.p.rapidapi.com/"

headers = {
	"X-RapidAPI-Key": "341580146bmsh6d7bf77c4ad811ep1d8340jsn825f96cd7f0a",
	"X-RapidAPI-Host": "imdb-top-100-movies.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
data =response.json()
data= pd.DataFrame(data)
data


rank                     title   
0      1  The Shawshank Redemption  \
1      2             The Godfather   
2      3           The Dark Knight   
3      4     The Godfather Part II   
4      5              12 Angry Men   
..   ...                       ...   
95    96            Reservoir Dogs   
96    97                     Ikiru   
97    98        Lawrence of Arabia   
98    99              Citizen Kane   
99   100                         M   

                                          description   
0   Two imprisoned men bond over a number of years...  \
1   The aging patriarch of an organized crime dyna...   
2   When the menace known as the Joker wreaks havo...   
3   The early life and career of Vito Corleone in ...   
4   The jury in a New York City murder trial is fr...   
..                                                ...   
95  When a simple jewelry heist goes horribly wron...   
96  A bureaucrat tries to find meaning in his life...   
97  The story of T.E. Lawrence, the English office...   
98  Following the death of publishing tycoon Charl...   
99  When the police in a German city are unable to...   

                                                image   
0   https://m.media-amazon.com/images/M/MV5BMDFkYT...  \
1   https://m.media-amazon.com/images/M/MV5BM2MyNj...   
2   https://m.media-amazon.com/images/M/MV5BMTMxNT...   
3   https://m.media-amazon.com/images/M/MV5BMWMwMG...   
4   https://m.media-amazon.com/images/M/MV5BMWU4N2...   
..                                                ...   
95  https://m.media-amazon.com/images/M/MV5BZmExNm...   
96  https://m.media-amazon.com/images/M/MV5BYWM1Ym...   
97  https://m.media-amazon.com/images/M/MV5BYWY5Zj...   
98  https://m.media-amazon.com/images/M/MV5BYjBiOT...   
99  https://m.media-amazon.com/images/M/MV5BODA4OD...   

                                            big_image   
0   https://m.media-amazon.com/images/M/MV5BMDFkYT...  \
1   https://m.media-amazon.com/images/M/MV5BM2MyNj...   
2   https://m.media-amazon.com/images/M/MV5BMTMxNT...   
3   https://m.media-amazon.com/images/M/MV5BMWMwMG...   
4   https://m.media-amazon.com/images/M/MV5BMWU4N2...   
..                                                ...   
95  https://m.media-amazon.com/images/M/MV5BZmExNm...   
96  https://m.media-amazon.com/images/M/MV5BYWM1Ym...   
97  https://m.media-amazon.com/images/M/MV5BYWY5Zj...   
98  https://m.media-amazon.com/images/M/MV5BYjBiOT...   
99  https://m.media-amazon.com/images/M/MV5BODA4OD...   

                            genre   
0                         [Drama]  \
1                  [Crime, Drama]   
2          [Action, Crime, Drama]   
3                  [Crime, Drama]   
4                  [Crime, Drama]   
..                            ...   
95              [Crime, Thriller]   
96                        [Drama]   
97  [Adventure, Biography, Drama]   
98               [Drama, Mystery]   
99     [Crime, Mystery, Thriller]   

                                            thumbnail rating      id  year   
0   https://m.media-amazon.com/images/M/MV5BMDFkYT...    9.3    top1  1994  \
1   https://m.media-amazon.com/images/M/MV5BM2MyNj...    9.2    top2  1972   
2   https://m.media-amazon.com/images/M/MV5BMTMxNT...    9.0    top3  2008   
3   https://m.media-amazon.com/images/M/MV5BMWMwMG...    9.0    top4  1974   
4   https://m.media-amazon.com/images/M/MV5BMWU4N2...    9.0    top5  1957   
..                                                ...    ...     ...   ...   
95  https://m.media-amazon.com/images/M/MV5BZmExNm...    8.3   top96  1992   
96  https://m.media-amazon.com/images/M/MV5BYWM1Ym...    8.3   top97  1952   
97  https://m.media-amazon.com/images/M/MV5BYWY5Zj...    8.3   top98  1962   
98  https://m.media-amazon.com/images/M/MV5BYjBiOT...    8.3   top99  1941   
99  https://m.media-amazon.com/images/M/MV5BODA4OD...    8.3  top100  1931   

       imdbid                             imdb_link  
0   tt0111161  https://www.imdb.com/title/tt0111161  
1 

In [60]:
import requests

url = "https://streaming-availability.p.rapidapi.com/search/title"

querystring = {"title":"dune : part two","country":"us","show_type":"all","output_language":"en"}

headers = {
	"X-RapidAPI-Key": "341580146bmsh6d7bf77c4ad811ep1d8340jsn825f96cd7f0a",
	"X-RapidAPI-Host": "streaming-availability.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data= response.json() 
data = pd.DataFrame(data)
data

result
0   {'type': 'movie', 'title': 'Dune', 'overview':...
1   {'type': 'movie', 'title': 'Fear Street: 1978'...
2   {'type': 'movie', 'title': 'Che: Part Two', 'o...
3   {'type': 'movie', 'title': 'The Cider House Ru...
4   {'type': 'movie', 'title': 'Sanshiro Sugata Pa...
5   {'type': 'movie', 'title': 'Ivan the Terrible,...
6   {'type': 'movie', 'title': 'A Chinese Odyssey ...
7   {'type': 'movie', 'title': 'Piranha II: The Sp...
8   {'type': 'movie', 'title': 'The Christmas Chro...
9   {'type': 'movie', 'title': 'Batman: The Long H...
10  {'type': 'movie', 'title': 'Olympia Part Two: ...
11  {'type': 'movie', 'title': 'A Genius, Two Frie...
12  {'type': 'movie', 'title': 'Dr. Mabuse, the Ga...
13  {'type': 'movie', 'title': 'My Piece of the Pi...
14  {'type': 'movie', 'title': 'A Piece of Sky', '...
15  {'type': 'movie', 'title': 'Underdog: Part Two...
16  {'type': 'movie', 'title': 'Knockback: 2', 'ov...
17  {'type': 'movie', 'title': 'Ah, Wilderness: Pa...
18  {'type': 'movie', 'title': 'Fist of Fury 2', '...
19  {'type': 'movie', 'title': 'The New Road: Akem...
20  {'type': 'movie', 'title': 'Zamfir's Zona Part...
21  {'type': 'movie', 'title': 'Corpse Prison: Par...
22  {'type': 'movie', 'title': 'We the Living, Par...
23  {'type': 'movie', 'title': 'Peter the First, P...
24  {'type': 'movie', 'title': 'A Bread Factory: P...

In [64]:
import requests

url = "https://advanced-movie-search.p.rapidapi.com/search/movie"

querystring = {"query":"Dune","page":"1"}

headers = {
	"X-RapidAPI-Key": "341580146bmsh6d7bf77c4ad811ep1d8340jsn825f96cd7f0a",
	"X-RapidAPI-Host": "advanced-movie-search.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data= response.json() 
data

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': 'https://image.tmdb.org/t/p/original/jYEW5xZkZk2WTrdbMGAPFuBqbDc.jpg',
   'genre_ids': [878, 12],
   'id': 438631,
   'original_language': 'en',
   'original_title': 'Dune',
   'overview': "Paul Atreides, a brilliant and gifted young man born into a great destiny beyond his understanding, must travel to the most dangerous planet in the universe to ensure the future of his family and his people. As malevolent forces explode into conflict over the planet's exclusive supply of the most precious resource in existence-a commodity capable of unlocking humanity's greatest potential-only those who can conquer their fear will survive.",
   'popularity': 420.499,
   'poster_path': 'https://image.tmdb.org/t/p/original/d5NXSklXo0qyIYkgV94XAgMIckC.jpg',
   'release_date': '2021-09-15',
   'title': 'Dune',
   'video': False,
   'vote_average': 7.788,
   'vote_count': 10149},
  {'adult': False,
   'backdrop_path': 'https://image.tmdb.org/t

In [67]:
import requests

url = "https://similar-movies.p.rapidapi.com/search"

querystring = {"q":"Dune"}

headers = {
	"X-RapidAPI-Key": "341580146bmsh6d7bf77c4ad811ep1d8340jsn825f96cd7f0a",
	"X-RapidAPI-Host": "similar-movies.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

response.json() 

{'movies:': [[{'movie': 'Dune (2021)', 'id': '65859-dune'},
   {'movie': 'Dune: Part Two (2024)', 'id': '90742-dune-part-two'},
   {'movie': 'Dune (2000)', 'id': '12573-dune'},
   {'movie': 'Dune (1984)', 'id': '12572-dune'},
   {'movie': "Frank Herbert's Children of Dune (2003)",
    'id': '10688-children-of-dune'},
   {'movie': 'Woman in the Dunes (1964)', 'id': '23474-woman-in-the-dunes'},
   {'movie': 'Long Road in the Dunes (1982)',
    'id': '37821-long-road-in-the-dunes'},
   {'movie': 'Dune Drifter (2020)', 'id': '71246-dune-drifter'},
   {'movie': 'Dune Warriors (1991)', 'id': '55194-dune-warriors'},
   {'movie': 'Planet Dune (2021)', 'id': '79725-planet-dune'}]]}

In [72]:
import requests

url = "https://mdblist.p.rapidapi.com/"

querystring = {"s":"batman"}

headers = {
	"X-RapidAPI-Key": "341580146bmsh6d7bf77c4ad811ep1d8340jsn825f96cd7f0a",
	"X-RapidAPI-Host": "mdblist.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


data= response.json() 
 
data

{'search': [{'id': 'tt0103359',
   'title': 'Batman: The Animated Series',
   'year': 1992,
   'score': 88,
   'score_average': 88,
   'type': 'show',
   'imdbid': 'tt0103359',
   'tmdbid': 2098,
   'traktid': 2085,
   'tvdbid': 76168},
  {'id': 'tt2313197',
   'title': 'Batman: The Dark Knight Returns, Part 1',
   'year': 2012,
   'score': 84,
   'score_average': 84,
   'type': 'movie',
   'imdbid': 'tt2313197',
   'tmdbid': 123025,
   'traktid': 84956},
  {'id': 'tt1569923',
   'title': 'Batman: Under the Red Hood',
   'year': 2010,
   'score': 84,
   'score_average': 84,
   'type': 'movie',
   'imdbid': 'tt1569923',
   'tmdbid': 40662,
   'traktid': 26756},
  {'id': 'tt0372784',
   'title': 'Batman Begins',
   'year': 2005,
   'score': 83,
   'score_average': 83,
   'type': 'movie',
   'imdbid': 'tt0372784',
   'tmdbid': 272,
   'traktid': 228},
  {'id': 'tt0118266',
   'title': 'The New Batman Adventures',
   'year': 1997,
   'score': 82,
   'score_average': 82,
   'type': 'show',


In [77]:
url =' http://api.themoviedb.org/3/movie/upcoming?api_key=b7cd3340a794e5a2f35e3abb820b497f'
response = requests.get(url)


data= response.json() 
 
data

{'dates': {'maximum': '2024-03-20', 'minimum': '2024-02-28'},
 'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/nTPFkLUARmo1bYHfkfdNpRKgEOs.jpg',
   'genre_ids': [35, 10749],
   'id': 1072790,
   'original_language': 'en',
   'original_title': 'Anyone But You',
   'overview': 'After an amazing first date, Bea and Ben’s fiery attraction turns ice cold — until they find themselves unexpectedly reunited at a destination wedding in Australia. So they do what any two mature adults would do: pretend to be a couple.',
   'popularity': 1589.046,
   'poster_path': '/yRt7MGBElkLQOYRvLTT1b3B1rcp.jpg',
   'release_date': '2023-12-21',
   'title': 'Anyone But You',
   'video': False,
   'vote_average': 6.89,
   'vote_count': 558},
  {'adult': False,
   'backdrop_path': '/meyhnvssZOPPjud4F1CjOb4snET.jpg',
   'genre_ids': [16, 28, 12, 35, 10751],
   'id': 940551,
   'original_language': 'en',
   'original_title': 'Migration',
   'overview': 'After a migrating duck family alights on thei

In [79]:
import requests
import pandas as pd

# Replace 'YOUR_API_KEY' with your actual TMDb API key
api_key = 'b7cd3340a794e5a2f35e3abb820b497f'
url = f"https://api.themoviedb.org/3/movie/upcoming?api_key={api_key}&language=en-US&page=1"

response = requests.get(url)
data = response.json()

# Assuming the response contains a list of movies in 'results'
movies = data.get('results', [])

# Convert to a DataFrame for easier viewing/manipulation
df = pd.DataFrame(movies)

# Display the first few rows to check
df

adult                     backdrop_path                   genre_ids   
0   False  /nTPFkLUARmo1bYHfkfdNpRKgEOs.jpg                 [35, 10749]  \
1   False  /meyhnvssZOPPjud4F1CjOb4snET.jpg     [16, 28, 12, 35, 10751]   
2   False  /oBIQDKcqNxKckjugtmzpIIOgoc4.jpg             [28, 53, 10752]   
3   False  /uF6ah2oXklyw3gU9ByWLKJ0tyfg.jpg                    [18, 53]   
4   False  /4woSOUD0equAYzvwhWBHIJDCM88.jpg                [28, 27, 53]   
5   False  /uUiIGztTrfDhPdAFJpr6m4UBMAd.jpg               [14, 28, 878]   
6   False  /uhUO7vQQKvCTfQWubOt5MAKokbL.jpg                   [878, 12]   
7   False   /zIYROrkHJPYB3VTiW1L9QVgaQO.jpg                    [28, 35]   
8   False  /jYEW5xZkZk2WTrdbMGAPFuBqbDc.jpg                   [878, 12]   
9   False  /bq9FpkTw9I3s1cSRuZxQibM2xOx.jpg             [18, 36, 10752]   
10  False  /ycnO0cjsAROSGJKuMODgRtWsHQw.jpg                    [18, 36]   
11  False  /bQS43HSLZzMjZkcHJz4fGc7fNdz.jpg            [878, 10749, 35]   
12  False  /bPe6svQylMx7zS4RyuMJbOL46ao.jpg                 [10402, 18]   
13  False  /dmiN2rakG9hZW04Xx7mHOoHTOyD.jpg                        [35]   
14  False  /2GzgongTSptjSkT7iCoXUcGIVB9.jpg             [10749, 18, 14]   
15  False   /7grv5ULlK9wMr2T1fkgr56HlbT.jpg             [18, 36, 10752]   
16  False  /hIeEpk2w1dnC9ly9ZJwwbunH2Cx.jpg  [16, 35, 18, 10751, 10402]   
17  False  /92mRLboU7VzQK1l4L0q9MOVGhbu.jpg                    [18, 36]   
18  False  /fOy2Jurz9k6RnJnMUMRDAgBwru2.jpg         [16, 10751, 35, 14]   
19  False  /fCwwzOziFYs7YRwP4gbrb9qH1xg.jpg     [28, 12, 16, 35, 10751]   

         id original_language        original_title   
0   1072790                en        Anyone But You  \
1    940551                en             Migration   
2    969492                en           Land of Bad   
3   1026436                en         Miller's Girl   
4   1096197                en             No Way Up   
5    634492                en            Madame Web   
6    693134                en        Dune: Part Two   
7    897087                en             Freelance   
8    438631                en                  Dune   
9    467244                en  The Zone of Interest   
10   872585                en           Oppenheimer   
11   792307                en           Poor Things   
12   802219                en  Bob Marley: One Love   
13   673593                en            Mean Girls   
14   994108                en   All of Us Strangers   
15   760774                en              One Life   
16   838240                es          Robot Dreams   
17   850165                en         The Iron Claw   
18   508947                en           Turning Red   
19  1011985                en       Kung Fu Panda 4   

                                             overview  popularity   
0   After an amazing first date, Bea and Ben’s fie...    1589.046  \
1   After a migrating duck family alights on their...     961.772   
2   When a Delta Force special ops mission goes te...     926.246   
3   A precocious young writer becomes involved wit...     674.800   
4   Characters from different backgrounds are thro...     530.547   
5   Forced to confront revelations about her past,...     499.642   
6   Follow the mythic journey of Paul Atreides as ...     427.280   
7   An ex-special forces operative takes a job to ...     424.040   
8   Paul Atreides, a brilliant and gifted young ma...     420.499   
9   The commandant of Auschwitz, Rudolf Höss, and ...     400.470   
10  The story of J. Robert Oppenheimer's role in t...     391.850   
11  Brought back to life by an unorthodox scientis...     378.631   
12  Jamaican singer-songwriter Bob Marley overcome...     321.532   
13  New student Cady Heron is welcomed into the to...     272.588   
14  One night in his near-empty tower block in con...     248.755   
15  British stockbroker Nicholas Winton visits Cze...     195.045   
16  Dog lives in Manhattan and he's tired of being...     167.759   
17  The true story of the inse

In [4]:
import requests
from datetime import datetime
import pandas as pd

def fetch_movies_released_in_month(api_key, year, month):
    base_url = "https://api.themoviedb.org/3"
    start_date = f"{year}-{month:02d}-01"
    end_date = f"{year}-{month:02d}-28"  # Simplification, adjust for months with 29, 30, or 31 days
    
    discover_url = f"{base_url}/discover/movie?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}"
    
    response = requests.get(discover_url)
    movies = response.json().get('results', [])
    
    for movie in movies:
        movie_id = movie['id']
        movie_details_url = f"{base_url}/movie/{movie_id}?api_key={api_key}"
        movie_details_response = requests.get(movie_details_url)
        movie_details = movie_details_response.json()
      
        print(movie)

# Example usage
api_key = 'b7cd3340a794e5a2f35e3abb820b497f'
year = 2024  # Example year
month = 3    # Example month (April)
df = fetch_movies_released_in_month(api_key, year, month)
df


{'adult': False, 'backdrop_path': '/veIyxxi5Gs8gvztLEW1Ysb8rrzs.jpg', 'genre_ids': [28, 878, 12], 'id': 823464, 'original_language': 'en', 'original_title': 'Godzilla x Kong: The New Empire', 'overview': 'Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', 'popularity': 262.339, 'poster_path': '/vkfuF8p5cm66aNB6sFmoPHrY3fE.jpg', 'release_date': '2024-03-27', 'title': 'Godzilla x Kong: The New Empire', 'video': False, 'vote_average': 0.0, 'vote_count': 0}
{'adult': False, 'backdrop_path': '/fCwwzOziFYs7YRwP4gbrb9qH1xg.jpg', 'genre_ids': [28, 12, 16, 35, 10751], 'id': 1011985, 'original_language': 'en', 'original_title': 'Kung Fu Panda 4', 'overview': 'Po is gearing up to become the spiritual leader of his Valley of Peace, but also needs someone to take his place as Dragon Warrior. As such, he will train a new kung fu practitioner for the spot and will encounte

In [5]:
import requests
import pandas as pd

def fetch_movies_released_in_month_to_df_with_posters(api_key, year, month):
    base_url = "https://api.themoviedb.org/3"
    image_base_url = "https://image.tmdb.org/t/p/w500"  # Base URL for images
    start_date = f"{year}-{month:02d}-01"
    end_date = f"{year}-{month:02d}-28"  # Simplification, adjust for accurate end dates
    
    discover_url = f"{base_url}/discover/movie?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}"
    
    response = requests.get(discover_url)
    movies = response.json().get('results', [])
    
    movie_details_list = []
    for movie in movies:
        movie_id = movie['id']
        movie_details_url = f"{base_url}/movie/{movie_id}?api_key={api_key}"
        movie_details_response = requests.get(movie_details_url)
        movie_details = movie_details_response.json()
        
        # Append the full URL for the poster image
        if movie_details.get('poster_path'):
            movie_details['poster_url'] = f"{image_base_url}{movie_details['poster_path']}"
            
        else:
            movie_details['over'] = None  # Handle movies without a poster
            
        movie_details['overview']= movie_details.get('overview')

        movie_details_list.append(movie_details)
    
    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(movie_details_list)
    return df

# Example usage
api_key = 'b7cd3340a794e5a2f35e3abb820b497f'
year = 2024  # Example year
month = 3    # Example month (March)
df = fetch_movies_released_in_month_to_df_with_posters(api_key, year, month)

# Display the DataFrame with the poster URL
print(df[['title', 'release_date', 'poster_url']])


                                      title release_date   
0           Godzilla x Kong: The New Empire   2024-03-27  \
1                           Kung Fu Panda 4   2024-03-06   
2               Ghostbusters: Frozen Empire   2024-03-21   
3                                Road House   2024-03-08   
4                                 Imaginary   2024-03-06   
5                                    Damsel   2024-03-08   
6        Winnie-the-Pooh: Blood and Honey 2   2024-03-23   
7                        Love Lies Bleeding   2024-03-08   
8                                Immaculate   2024-03-20   
9                                       Eks   2024-03-01   
10                   The Thundermans Return   2024-03-07   
11           Megamind vs the Doom Syndicate   2024-03-01   
12                          Arthur the King   2024-03-15   
13                               Irish Wish   2024-03-15   
14                            Weeding Night   2024-03-07   
15                              Problemi

In [2]:
import requests
import pandas as pd
from calendar import monthrange
import datetime

def fetch_current_month_movies_to_df_with_posters(api_key):
    base_url = "https://api.themoviedb.org/3"
    image_base_url = "https://image.tmdb.org/t/p/w500"  # Base URL for images
    
    # Use datetime to get the current year and month
    current_date = datetime.datetime.now()
    year = current_date.year
    month = current_date.month +1
    
    start_date = f"{year}-{month:02d}-01"
    end_of_month_day = monthrange(year, month)[1]  # Dynamically get the end day of the month
    end_date = f"{year}-{month:02d}-{end_of_month_day}"  # Accurate end date
    
    discover_url = f"{base_url}/discover/movie?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}"
    
    response = requests.get(discover_url)
    movies = response.json().get('results', [])
    
    movie_details_list = []
    for movie in movies:
        movie_id = movie['id']
        movie_details_url = f"{base_url}/movie/{movie_id}?api_key={api_key}"
        movie_details_response = requests.get(movie_details_url)
        movie_details = movie_details_response.json()
        
        # Construct a simplified dictionary for each movie
        simplified_details = {
            'title': movie_details.get('title'),
            'release_date': movie_details.get('release_date'),
            'poster_url': f"{image_base_url}{movie_details['poster_path']}" if movie_details.get('poster_path') else None,
            'overview': movie_details.get('overview'),
            'genre': movie_details.get('genre')
            
            
        }
        
        movie_details_list.append(simplified_details)
    
    # Convert the list of simplified dictionaries into a DataFrame
    df = pd.DataFrame(movie_details_list)
    return df

# Example usage
api_key = 'b7cd3340a794e5a2f35e3abb820b497f'
df = fetch_current_month_movies_to_df_with_posters(api_key)

# Display the DataFrame with the poster URL
print(df[['title', 'release_date', 'poster_url', 'overview']])


                                                title release_date   
0                                             Abigail   2024-04-18  \
1                Rebel Moon - Part Two: The Scargiver   2024-04-19   
2                                           Civil War   2024-04-11   
3                                         Challengers   2024-04-24   
4                                        The Fall Guy   2024-04-25   
5               The Ministry of Ungentlemanly Warfare   2024-04-18   
6                                     Boy Kills World   2024-04-12   
7                                      The First Omen   2024-04-03   
8                                       Back to Black   2024-04-11   
9                                          Monkey Man   2024-04-04   
10                            How to Date Billy Walsh   2024-04-05   
11                                      Jericho Ridge   2024-04-25   
12                                              Sting   2024-04-12   
13                  

In [6]:
base_url = "https://api.themoviedb.org/3"
image_base_url = "https://image.tmdb.org/t/p/w500"  # Base URL for images
    
current_date = datetime.datetime.now()
year = current_date.year
month = current_date.month +1

start_date = f"{year}-{month:02d}-01"
end_of_month_day = monthrange(year, month)[1]
end_date = f"{year}-{month:02d}-{end_of_month_day}"

discover_url = f"{base_url}/discover/movie?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}"

response = requests.get(discover_url)

movies = response.json().get('results', [])

movie_details_list = []
for movie in movies:
    poster_url = f"{image_base_url}{movie['poster_path']}" if movie.get('poster_path') else None
    
    movie_details_list.append({
        'title': movie.get('title'),
        'release_date': movie.get('release_date'),
        'poster_url': poster_url,
        'overview': movie.get('overview')
    })

df = pd.DataFrame(movie)

AttributeError: module 'datetime' has no attribute 'now'

In [7]:
df

title release_date   
0      Godzilla x Kong: The New Empire   2024-03-27  \
1                      Kung Fu Panda 4   2024-03-06   
2                           Road House   2024-03-08   
3                            Imaginary   2024-03-06   
4          Ghostbusters: Frozen Empire   2024-03-21   
5                                  Eks   2024-03-01   
6                     A Frozen Rooster   2024-03-29   
7                               Damsel   2024-03-08   
8   Winnie-the-Pooh: Blood and Honey 2   2024-03-23   
9                           Immaculate   2024-03-20   
10                  Love Lies Bleeding   2024-03-08   
11              The Thundermans Return   2024-03-07   
12                            Stanczyk   2024-03-01   
13                       Weeding Night   2024-03-07   
14      Megamind vs the Doom Syndicate   2024-03-01   
15                     Arthur the King   2024-03-15   
16                             Kapalit   2024-03-08   
17                  Gump – jsme dvojka   2024-03-28   
18                          Irish Wish   2024-03-15   
19                         Problemista   2024-03-01   

                                           poster_url   
0   https://image.tmdb.org/t/p/w500/bQ2ywkchIiaKLS...  \
1   https://image.tmdb.org/t/p/w500/kDp1vUBnMpe8ak...   
2   https://image.tmdb.org/t/p/w500/bXi6IQiQDHD00J...   
3   https://image.tmdb.org/t/p/w500/9u6HEtZJdZDjPG...   
4   https://image.tmdb.org/t/p/w500/86OcNR3xr7JknB...   
5   https://image.tmdb.org/t/p/w500/uyde7bBL7rr5Sq...   
6   https://image.tmdb.org/t/p/w500/Qotf3QgDqUZEW6...   
7   https://image.tmdb.org/t/p/w500/sMp34cNKjIb18U...   
8   https://image.tmdb.org/t/p/w500/2sADrLwMQof6yY...   
9   https://image.tmdb.org/t/p/w500/fdZpvODTX5wwkD...   
10  https://image.tmdb.org/t/p/w500/p7jyFWiLyHPttq...   
11  https://image.tmdb.org/t/p/w500/hUMMTkiFK6wiv2...   
12  https://image.tmdb.org/t/p/w500/dcxWZEvmhn1L9V...   
13  https://image.tmdb.org/t/p/w500/562Feolv3s54cJ...   
14  https://image.tmdb.org/t/p/w500/yRZfiG1QpRkBc7...   
15  https://image.tmdb.org/t/p/w500/kzTMaDxz69yQ2p...   
16  https://image.tmdb.org/t/p/w500/zOI7WTJYcAsBSS...   
17  https://image.tmdb.org/t/p/w500/crBSk1umI2Ma0A...   
18  https://image.tmdb.org/t/p/w500/v4Bb70dpIIQoEn...   
19  https://image.tmdb.org/t/p/w500/wDnqUnU657BLda...   

                                             overview  
0   Following their explosive showdown, Godzilla a...  
1   Po is gearing up to become the spiritual leade...  
2   Ex-UFC fighter Dalton takes a job as a bouncer...  
3   When Jessica moves back into her childhood hom...  
4   The Spengler family returns to where it all st...  
5   Three compelling stories in one erotic experie...  
6   In the final Huevos adventure, Toto and his fa...  
7   A dutiful damsel agrees to marry a handsome pr...  
8   Winnie the Pooh, Piglet, Owl, and Tigger decid...  
9   Cecilia, a woman of devout faith, is offered a...  
10  Reclusive gym manager Lou falls hard for Jacki...  
11  Twins Phoebe and Max are enjoying their superh...  
12  'A Sheep Will Follow'\r An Impressionable teen...  
13  Lucía and Nico, the couple of the century, are...  
14  Megamind’s former villain team, The Doom Syndi...  
15  Over the course of ten days and 435 miles, an ...  
16                                       Details here  
17                                                     
18  When the love of her life gets engaged to her ...  
19  Alejandro is an aspiring toy designer from El ...

In [7]:
TMDB_API_KEY = 'b7cd3340a794e5a2f35e3abb820b497f'
TMDB_BASE_URL = "https://api.themoviedb.org/3"

def get_movie_id_by_title(movie_title: str) -> int:
    """
    Fetches the TMDB ID for a movie by title.
    
    :param movie_title: The title of the movie for which to search.
    :return: The TMDB ID of the movie, or None if not found.
    """
    search_url = f"{TMDB_BASE_URL}/search/movie"
    params = {
        'api_key': TMDB_API_KEY,
        'query': movie_title
    }
    
    response = requests.get(search_url, params=params)
    response_json = response.json()
    
    # Assuming the first result is the best match
    results = response_json.get('results')
    if results:
        # Return the ID of the first movie in the results
        return results[0]
    else:
        return None

# Example usage

movie_id = get_movie_id_by_title("Avatar")
movie_id

NameError: name 'requests' is not defined

In [6]:
def get_similar_movies(movie_id: int):
    similar_movies_url = f"{TMDB_BASE_URL}/movie/{movie_id}/similar"
    response = requests.get(similar_movies_url, params={"api_key": TMDB_API_KEY})
    results = response.json().get("results", [])
    return results
movies = get_similar_movies(19995)
movies

[{'adult': False,
  'backdrop_path': '/m0edcSYyX4A4EgzsI9wtXIPsV6.jpg',
  'genre_ids': [28, 12, 18, 36, 10752],
  'id': 14111,
  'original_language': 'en',
  'original_title': 'Battle for Haditha',
  'overview': 'An investigation of the massacre of 24 men, women and children in Haditha, Iraq allegedly shot by 4 U.S. Marines in retaliation for the death of a U.S. Marine killed by a roadside bomb. The movie follows the story of the Marines of Kilo Company, an Iraqi family, and the insurgents who plant the roadside bomb.',
  'popularity': 11.442,
  'poster_path': '/yUfXGuipKIw2xKs64yrTPOwRTtd.jpg',
  'release_date': '2007-09-11',
  'title': 'Battle for Haditha',
  'video': False,
  'vote_average': 6.313,
  'vote_count': 112},
 {'adult': False,
  'backdrop_path': '/cepDjUrzKRa0WqcuSnWsmajWbo8.jpg',
  'genre_ids': [16, 878, 10752, 28],
  'id': 14132,
  'original_language': 'ja',
  'original_title': '超時空要塞マクロス 愛・おぼえていますか',
  'overview': 'It is A.D. 2009 and the human race is caught in a war 

In [6]:
import tmdbsimple as tmdb
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
tmdb.API_KEY ='b7cd3340a794e5a2f35e3abb820b497f'

from requests.exceptions import HTTPError
import tmdbsimple as tmdb
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def search_movies(query):
    search = tmdb.Search()
    response = search.movie(query=query)
    movies = []
    for result in response:
        movie_dict = {
            'title': result.title,
            'genre_ids': result.genre_ids,
            'overview': result.overview  # May include for further filtering/processing
        }
        movies.append(movie_dict)
    return pd.DataFrame(movies)


def get_movie_details(movie_id, api_key=tmdb.API_KEY):
    movie = tmdb.Movies(api_key)
    details = movie.details(movie_id)
    return {
        'title': details.title,
        'genres': [genre['name'] for genre in details.genres],  # Extract genre names
        'average_rating': details.vote_average,
        'vote_count': details.vote_count
    }

def clean_text(text):
    # Implement text cleaning logic here (e.g., lowercase, remove punctuation/stop words)
    return text.lower().strip()




In [41]:
user_movie = input("Enter a movie title: ")
user_movie_df = search_movies(user_movie)

if user_movie_df.empty:
    print("No movies found for your search.")
else:
    # Choose the desired movie from search results (if multiple matches exist)
    user_movie_id = user_movie_df['id'].iloc[0]  # Assuming the first match is chosen

    all_movies_df = search_movies('popular').merge(  # Retrieve popular movies initially
        get_movie_details(user_movie_id)['genres'],  # Merge additional details for user movie
        left_on='genre_ids',
        right_on='genres',  # Merge based on shared genres
        how='inner'  # Only keep movies with shared genres
    )



    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(all_movies_df['overview'])
    user_movie_vector = tfidf_matrix[all_movies_df[all_movies_df['title'] == user_movie]['index'].iloc[0]]

    cosine_similarities = cosine_similarity(user_movie_vector, tfidf_matrix)
    sorted_similar_movies = all_movies_df.iloc[cosine_similarities.argsort()[::-1]]

    top_n_recommendations = sorted_similar_movies.head(10)  # Select top 10 recommendations
    print("\nTop 10 Movie Recommendations:")
    for i, row in top_n_recommendations.iterrows():
        print(f"{i+1}. {row['title']}")
        print(f"   - Genres: {', '.join(row['genres'])}")
        print(f"   - Average Rating: {row['average_rating']:.1f} ({row['vote_count']} votes)")
        print()


AttributeError: 'str' object has no attribute 'genre_ids'

In [47]:
all_movies_df = search_movies('popular')
all_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   page           20 non-null     int64 
 1   results        20 non-null     object
 2   total_pages    20 non-null     int64 
 3   total_results  20 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 772.0+ bytes


In [48]:
import pandas as pd
import tmdbsimple as tmdb
def search_movies(query):
    search = tmdb.Search()
    response = search.movie(query=query)
    movies = []
    for result in response['results']:
        movies.append({
            'id': result['id'],
            'title': result['title'],
            'overview': result['overview']
        })
    return pd.DataFrame(movies)

def get_movie_details(movie_ids):
    details_list = []
    for movie_id in movie_ids:
        movie = tmdb.Movies(movie_id)
        details = movie.info()
        details_list.append({
            'id': details['id'],
            'title': details['title'],
            'overview': details['overview']
        })
    return pd.DataFrame(details_list)


In [53]:
def get_popular_movies(n=10000):
    movie = tmdb.Movies()
    popular = movie.popular(page=1)
    movies = popular['results'][:n]
    return get_movie_details([m['id'] for m in movies])


In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recommend_movies(user_movie_title):
    user_movie_df = search_movies(user_movie_title)
    if user_movie_df.empty:
        print("No movies found for your search.")
        return
    
    popular_movies_df = get_popular_movies(n=10000)
    
    # Combine user movie with popular movies for vectorization
    combined_df = pd.concat([user_movie_df, popular_movies_df]).drop_duplicates().reset_index(drop=True)
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_df['overview'].fillna(''))
    
    # Assuming user's movie is the first entry
    user_movie_vector = tfidf_matrix[0]
    
    cosine_sim = cosine_similarity(user_movie_vector, tfidf_matrix)
    similarity_scores = list(enumerate(cosine_sim[0]))
    
    # Sort movies based on similarity scores
    sorted_similar_movies = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get top 10 recommendations, skipping the first since it's the user's movie
    top_recommendations = sorted_similar_movies[1:11]
    
    print("\nTop 10 Movie Recommendations:")
    for i, (index, score) in enumerate(top_recommendations, start=1):
        print(f"{i}. {combined_df.iloc[index]['title']} (Score: {score:.2f})")


In [59]:
user_movie_title = input("Enter a movie title: ")
recommend_movies(user_movie_title)


Top 10 Movie Recommendations:
1. Land of Bad (Score: 0.07)
2. Avatar (Score: 0.00)
3. Avatar: The Way of Water (Score: 0.00)
4. Avatar: Creating the World of Pandora (Score: 0.00)
5. The King's Avatar: For the Glory (Score: 0.00)
6. Avataro Sentai Donbrothers vs. Zenkaiger (Score: 0.00)
7. Avatar 3 (Score: 0.00)
8. The Botanical Avatar of Mademoiselle Flora (Score: 0.00)
9. Avatar 4 (Score: 0.00)
10. Avatar: The Deep Dive - A Special Edition of 20/20 (Score: 0.00)


In [60]:
import pandas as pd

def search_movies(query):
    search = tmdb.Search()
    response = search.movie(query=query)
    movies = []
    for result in response['results']:
        movies.append({
            'id': result['id'],
            'title': result['title'],
            'overview': result['overview']
        })
    return pd.DataFrame(movies)

def get_movie_details(movie_ids):
    details_list = []
    for movie_id in movie_ids:
        movie = tmdb.Movies(movie_id)
        details = movie.info()
        details_list.append({
            'id': details['id'],
            'title': details['title'],
            'overview': details['overview']
        })
    return pd.DataFrame(details_list)


In [65]:
def get_user_movies():
    movie_titles = []
    while True:
        title = input("Enter a movie title (or type 'done' when finished): ")
        if title.lower() == 'done':
            break
        movie_titles.append(title)
    
    selected_movies = []
    for title in movie_titles:
        print(f"\nSearching for '{title}'...")
        search_results = search_movies(title)
        selected_movie = user_select_movie(search_results)
        if selected_movie is not None:
            selected_movies.append(selected_movie)
        else:
            print(f"Could not find an exact match for '{title}'. Moving on to the next title.")
    
    return selected_movies


In [64]:
def user_select_movie(movies_df):
    if movies_df.empty:
        print("No movies found for your search.")
        return None
    elif len(movies_df) == 1:
        return movies_df.iloc[0]
    else:
        print("\nMultiple movies found. Please select the correct one:")
        for index, row in movies_df.iterrows():
            print(f"{index + 1}. {row['title']} ({row.get('release_date', 'N/A')})")
        choice = int(input("\nEnter the number of the correct movie: ")) - 1
        if choice >= 0 and choice < len(movies_df):
            return movies_df.iloc[choice]
        else:
            print("Invalid selection.")
            return None


In [ ]:
def recommend_movies_for_multiple_titles(movie_titles):
    all_user_movies = pd.DataFrame()
    for title in movie_titles:
        user_movie_df = search_movies(title)
        user_movie = user_select_movie(user_movie_df)
        if user_movie is not None:
            all_user_movies = pd.concat([all_user_movies, pd.DataFrame([user_movie])])
    
    if all_user_movies.empty:
        print("No valid movies were selected.")
        return

    # Assuming get_popular_movies() is defined to fetch a larger set of popular movies
    popular_movies_df = get_popular_movies(n=100)
    combined_df = pd.concat([all_user_movies, popular_movies_df]).drop_duplicates().reset_index(drop=True)
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_df['overview'].fillna(''))
    
    # Compute average similarity scores across all user movies
    avg_cosine_sim = np.zeros(tfidf_matrix.shape[0])
    for index in all_user_movies.index:
        cosine_sim = cosine_similarity(tfidf_matrix[index], tfidf_matrix)
        avg_cosine_sim += cosine_sim.flatten()
    avg_cosine_sim /= len(all_user_movies)
    
    similarity_scores = list(enumerate(avg_cosine_sim))
    sorted_similar_movies = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get top recommendations, excluding the user's movies themselves
    top_recommendations_indices = [item[0] for item in sorted_similar_movies[1:11] if item[0] not in all_user_movies.index]
    
    print("\nTop Movie Recommendations:")
    for i, index in enumerate(top_recommendations_indices, start=1):
        print(f"{i}. {combined_df.iloc[index]['title']} (Score: {avg_cosine_sim[index]:.2f})")


In [68]:
movie_titles = get_user_movies()
recommend_movies_for_multiple_titles(movie_titles)



Searching for 'avenger'...

Multiple movies found. Please select the correct one:
1. Avenger (N/A)
2. Avengers: Infinity War (N/A)
3. Avengers: Endgame (N/A)
4. Avengers Confidential: Black Widow & Punisher (N/A)
5. The Avengers (N/A)
6. Avengers: Age of Ultron (N/A)
7. Captain America: The First Avenger (N/A)
8. Crippled Avengers (N/A)
9. Avengers 5 (N/A)
10. LEGO Marvel Avengers: Code Red (N/A)
11. The Avengers (N/A)
12. Masked Avengers (N/A)
13. Avengers: Secret Wars (N/A)
14. Next Avengers: Heroes of Tomorrow (N/A)
15. The Avenger (N/A)
16. The Masked Avenger: Lagardere (N/A)
17. Avengers Grimm (N/A)
18. Avengers Grimm: Time Wars (N/A)
19. The Toxic Avenger (N/A)
20. Ultimate Avengers: The Movie (N/A)

Searching for 'batman'...

Multiple movies found. Please select the correct one:
1. Batman (N/A)
2. Batman (N/A)
3. Batman (N/A)
4. Batman (N/A)
5. The Batman (N/A)
6. Batman Ninja (N/A)
7. Batman v Superman: Dawn of Justice (N/A)
8. Batman Begins (N/A)
9. Scooby-Doo! & Batman: The 

ValueError: invalid literal for int() with base 10: ''

In [25]:
 
from sklearn.metrics.pairwise import linear_kernel
import tmdbsimple as tmdb
from requests.exceptions import HTTPError
import tmdbsimple as tmdb
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


tmdb.API_KEY ='b7cd3340a794e5a2f35e3abb820b497f'


def search_movies(query):
    search = tmdb.Search()
    response = search.movie(query=query)
    movies = []
    for result in response['results']:
        movie_dict = {
            'id': result['id'],
            'title': result['title'],
            'overview': result['overview']
        }
        movies.append(movie_dict)
    return pd.DataFrame(movies)

def get_movie_details(movie_id):
    movie = tmdb.Movies(movie_id)
    details = movie.info()
    base_image_url = "https://image.tmdb.org/t/p/w500"  # Using 'w500' as an example size
    poster_url = f"{base_image_url}{details['poster_path']}" if details.get('poster_path') else None

    return {
        'id': details['id'],
        'title': details['title'],
        'genres': [genre['name'] for genre in details['genres']],
        'overview': details['overview'],
        'poster_url': poster_url  # Add the poster URL to your dictionary
    }

def get_recommendations(movie_title, num_recommendations=2):
    df = search_movies(movie_title)
    if df.empty:
        print(f"No movies found for {movie_title}.")
        return pd.DataFrame()

    movie_details = []
    for index, row in df.iterrows():
        details = get_movie_details(row['id'])
        movie_details.append(details)
    movies_df = pd.DataFrame(movie_details)

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(movies_df['overview'].fillna(''))
    
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]

    movie_indices = [i[0] for i in sim_scores]

    return movies_df.iloc[movie_indices]

def main():
    movie_titles = ["Inception", "The Matrix", "batman", "avatar"]
    all_recommendations = pd.DataFrame()
    
    for title in movie_titles:
        recommendations = get_recommendations(title)
        all_recommendations = pd.concat([all_recommendations, recommendations], ignore_index=True)

    # Remove potential duplicates
    all_recommendations.drop_duplicates(subset=['id'], inplace=True)

    if not all_recommendations.empty:
        print("\nFinal Recommendations:")
        for index, row in all_recommendations.iterrows():
            print(f"{index+1}: {row['title']} - {', '.join(row['genres'])}")
    else:
        print("No recommendations available.")

if __name__ == "__main__":
    main()



Final Recommendations:
1: Inception: The Cobol Job - Animation, Action, Thriller, Science Fiction
2: Inception: Music from the Motion Picture - Music
3: Making 'Enter the Matrix' - Documentary, Action, Science Fiction
4: Return to Source: The Philosophy of The Matrix - Documentary
5: Batman Returns - Action, Fantasy
6: Batman Begins - Action, Crime, Drama
7: Avatar - Drama
8: Avatar: The Way of Water - Science Fiction, Adventure, Action


In [28]:
from fastapi import FastAPI, HTTPException, Request, Depends, Body
from typing import List, Optional
from pydantic import BaseModel

In [8]:
movie_titles = ["Inception"]
def get_movie_recommendations(movie_titles: List[str], num_recommendations: int = 3):
    all_recommendations = pd.DataFrame()
    
    if len(movie_titles) == 0:
        raise HTTPException(status_code=400, detail="No movie titles provided.")
    
    # Adjust the number of recommendations based on the number of input movies
    if len(movie_titles) <= 2:
        num_rec = 5
    else:
        num_rec = num_recommendations

    for title in movie_titles:
        recommendations = get_recommendations(title, num_recommendations=num_rec)
        all_recommendations = pd.concat([all_recommendations, recommendations], ignore_index=True)

    # Remove potential duplicates
    all_recommendations.drop_duplicates(subset=['title'], inplace=True)

    if all_recommendations.empty:
        return {"message": "No recommendations available."}
    
    recommendations_list = all_recommendations.to_dict('records')
    return {"moviesr": recommendations_list}


In [11]:
get_movie_recommendations(movie_titles, 3)

{'moviesr': [{'id': 64956,
   'title': 'Inception: The Cobol Job',
   'genres': ['Animation', 'Action', 'Thriller', 'Science Fiction'],
   'overview': 'This "Inception" prequel unfolds courtesy of a beautiful Motion Comic, and explains how Cobb, Arthur and Nash were enlisted by Cobol Engineering.',
   'average_rating': 0.7911524587051451,
   'vote_count': 0.008408119355660618},
  {'id': 542438,
   'title': 'Bikini Inception',
   'genres': ['Comedy'],
   'overview': "Two flunky Janitors in an Arctic Lab perform unauthorized experiments transporting them to a beach dream world in Malibu California w/50 beautiful young girls and a female Brazilian PhD Student wearing only a bra and panties. A '67 Muscle car races chases horses guns fights surfing, sumo wrestler, wolf monster, underwater scenes tons of gorgeous models. Sexy sci-fi fun.",
   'average_rating': 0.7404594645908484,
   'vote_count': 0.0},
  {'id': 613092,
   'title': 'The Crack: Inception',
   'genres': ['Drama', 'Thriller'],
 

In [22]:
movie_id = 1360
def get_movie_details(movie_id):
    movie = tmdb.Movies(movie_id)
    details = movie.info()
    base_image_url = "https://image.tmdb.org/t/p/w500"  # Using 'w500' as an example size
    poster_url = f"{base_image_url}{details['poster_path']}" if details.get('poster_path') else None
    return {
        'id': details['id'],
        'title': details['title'],
        'genres': [genre['name'] for genre in details['genres']],
        'overview': details['overview'],
        'average_rating': details['vote_average'],
        'vote_count': details['vote_count'],
        'popularity': details['popularity'],
        'poster': poster_url
    }
get_movie_details(movie_id)

{'id': 1360,
 'title': 'Frida',
 'genres': ['Drama', 'Romance'],
 'overview': 'A biography of artist Frida Kahlo, who channeled the pain of a crippling injury and her tempestuous marriage into her work.',
 'average_rating': 7.421,
 'vote_count': 1939,
 'popularity': 14.611,
 'poster': 'https://image.tmdb.org/t/p/w500/a4hgR6aKoohB6MHni171jbi9BkU.jpg'}

In [31]:
import requests
TMDB_API_KEY = 'b7cd3340a794e5a2f35e3abb820b497f'
TMDB_BASE_URL = "https://api.themoviedb.org/3"
def get_movie_id_by_title(movie_title: str) -> int:
    search_url = f"{TMDB_BASE_URL}/search/movie"
    params = {
        'api_key': TMDB_API_KEY,
        'query': movie_title
    }
    
    response = requests.get(search_url, params=params)
    response_json = response.json()
    
    # Assuming the first result is the best match
    results = response_json.get('results')
    if results:
        # Return the ID of the first movie in the results
        return results
    else:
        return None
get_movie_id_by_title("avengers")

[{'adult': False,
  'backdrop_path': '/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg',
  'genre_ids': [12, 28, 878],
  'id': 299536,
  'original_language': 'en',
  'original_title': 'Avengers: Infinity War',
  'overview': 'As the Avengers and their allies have continued to protect the world from threats too large for any one hero to handle, a new danger has emerged from the cosmic shadows: Thanos. A despot of intergalactic infamy, his goal is to collect all six Infinity Stones, artifacts of unimaginable power, and use them to inflict his twisted will on all of reality. Everything the Avengers have fought for has led up to this moment - the fate of Earth and existence itself has never been more uncertain.',
  'popularity': 206.337,
  'poster_path': '/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg',
  'release_date': '2018-04-25',
  'title': 'Avengers: Infinity War',
  'video': False,
  'vote_average': 8.249,
  'vote_count': 28540},
 {'adult': False,
  'backdrop_path': '/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg',
  'genre_id

In [33]:
def get_similar_movies(movie_title: str) -> int:
    search_url = f"{TMDB_BASE_URL}/search/movie"
    params = {
        'api_key': TMDB_API_KEY,
        'query': movie_title
    }
    
    response = requests.get(search_url, params=params)
    response_json = response.json()
    
    # Assuming the first result is the best match
    results = response_json.get('results')
    if results:
        # Return the ID of the first movie in the results
        return results
    else:
        return None

get_similar_movies(f'The Shawshank Redemption')


[{'adult': False,
  'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg',
  'genre_ids': [18, 80],
  'id': 278,
  'original_language': 'en',
  'original_title': 'The Shawshank Redemption',
  'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.',
  'popularity': 128.965,
  'poster_path': '/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg',
  'release_date': '1994-09-23',
  'title': 'The Shawshank Redemption',
  'video': False,
  'vote_average': 8.7,
  'vote_count': 25691},
 {'adult': False,
  'backdrop_path': None,
  'genre_ids': [],
  'id': 897459,
  'original_language': 'en',
  'original_title': 'Hope Springs Eternal: A Look Back at The Shawshank Redemption',


In [16]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from typing import List
import httpx
async def get_suggestions() -> List[str]:
    url = "https://example-imdb-api.com/search"
    params = {
        "api_key": "b7cd3340a794e5a2f35e3abb820b497f"
    }
    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            # Assuming the API returns a list of titles in the search results
            return [movie['title'] for movie in data['results']]
        else:
            raise HTTPException(status_code=404, detail="Suggestions not found")

In [39]:
get_suggestions()

<coroutine object get_suggestions at 0x7f40c0d8fe40>

In [2]:
title = "avatar"

In [4]:
movie_details= [title]
movie_details

['avatar']

In [13]:
from fastapi import FastAPI, HTTPException
from typing import List
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
import tmdbsimple as tmdb
import random
api_key ='b7cd3340a794e5a2f35e3abb820b497f'

In [15]:
movie_titles = ["avatar" , "Batman" , "The Matrix", "avengers"]
def read_root(request: Request, title: Optional[str] = None, db: Session = Depends(get_db), q: str = "", num_recommendations: int = 3):
    user_id = request.session.get('user_id', None)
    user = None
    if user_id:
        user = get_user(db, user_id=user_id)
    
    # Movie fetching
    upcoming_df = fetch_movies(api_key, "upcoming")
    now_playing_df = fetch_movies(api_key, "now_playing")
    
    upcoming_movies = upcoming_df.to_dict(orient="records") if not upcoming_df.empty else []
    now_playing_movies = now_playing_df.to_dict(orient="records") if not now_playing_df.empty else []
   


    all_recommendations = pd.DataFrame()
    
    if len(movie_titles) == 0:
        raise HTTPException(status_code=400, detail="No movie titles provided.")
    
    # Adjust the number of recommendations based on the number of input movies
    if len(movie_titles) <= 2:
        num_rec = 5
    else:
        num_rec = num_recommendations

    for title in movie_titles:
        recommendations = get_recommendations(title, num_recommendations=num_rec)
        all_recommendations = pd.concat([all_recommendations, recommendations], ignore_index=True)

    # Remove potential duplicates
    all_recommendations.drop_duplicates(subset=['title'], inplace=True)

    if all_recommendations.empty:
        return {"message": "No recommendations available."}
    
    recommendations_list = all_recommendations.to_dict('records')
    random.shuffle(recommendations_list)


    # Combined template response
    return templates.TemplateResponse("index.html", {
        "request": request,
        "user": user,
        "movies": recommendations_list,
        "now_movies": now_playing_movies         
    })

NameError: name 'Depends' is not defined

In [35]:
def search_movies(query):
    url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={query}'
    response = requests.get(url)
    movies = []
    for result in response['results']:
        movie_dict = {
            'id': result['id'],
            'title': result['title'],
            'overview': result['overview']
        }
        movies.append(movie_dict)
    return pd.DataFrame(movies)

In [27]:
def get_movie_details(movie_id):
    movie = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}'
    response = requests.get(url)
    df = response.json()
    details = movie.info()
    base_image_url = "https://image.tmdb.org/t/p/w500"  # Using 'w500' as an example size
    poster_url = f"{base_image_url}{details['poster_path']}"
    return {
        'id': details['id'],
        'title': details['title'],
        'genres': [genre['name'] for genre in details['genres']],
        'overview': details['overview'],
        'average_rating': details['vote_average'],
        'vote_count': details['vote_count'],
        'popularity': details['popularity'],
        'poster': poster_url
         }

In [28]:
def get_recommendations(movie_title, num_recommendations=3):
    df = search_movies(movie_title)
    url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={query}'
    response = requests.get(url)
    df = response.json()
    if df.empty:
        print(f"No movies found for {movie_title}.")
        return pd.DataFrame()

    movie_details = []
    for index, row in df.iterrows():
        details = get_movie_details(row['id'])
        movie_details.append(details)
    movies_df = pd.DataFrame(movie_details)

    # Normalize the numeric features
    scaler = MinMaxScaler()
    movies_df[['average_rating', 'vote_count']] = scaler.fit_transform(movies_df[['average_rating', 'vote_count']])
    
    # Process textual data
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(movies_df['overview'].fillna(''))
    
    # Combine features
    combined_features = np.hstack((tfidf_matrix.toarray(), movies_df[['average_rating', 'vote_count']].to_numpy()))
    
    # Compute similarity
    cosine_sim = linear_kernel(combined_features, combined_features)

    sim_scores = list(enumerate(cosine_sim[0]))
     
    
    sim_scores = sim_scores[1:num_recommendations+1]

    movie_indices = [i[0] for i in sim_scores]

    return movies_df.iloc[movie_indices]

In [29]:
def get_movie_recommendations(movie_titles, num_recommendations = 3):
    all_recommendations = pd.DataFrame()
    
    if len(movie_titles) == 0:
        raise HTTPException(status_code=400, detail="No movie titles provided.")
    
    # Adjust the number of recommendations based on the number of input movies
    if len(movie_titles) <= 2:
        num_rec = 5
    else:
        num_rec = num_recommendations

    for title in movie_titles:
        recommendations = get_recommendations(title, num_recommendations)
    
        all_recommendations = pd.concat([all_recommendations, recommendations], ignore_index=True)

    # Remove potential duplicates
    all_recommendations.drop_duplicates(subset=['title'], inplace=True)

    if all_recommendations.empty:
        return {"message": "No recommendations available."}
    
    recommendations_list = all_recommendations.to_dict('records')
    return {"recommendations": recommendations_list}


In [36]:
import requests
get_movie_recommendations("the matrix" , 3)

TypeError: 'Response' object is not subscriptable

In [40]:
from typing import List
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
import tmdbsimple as tmdb
import random
app = FastAPI()

tmdb.API_KEY ='b7cd3340a794e5a2f35e3abb820b497f'
movie_titles = ["Inception"]
def search_movies(query):
    search = tmdb.Search()
    response = search.movie(query=query)
    movies = []
    for result in response['results']:
        movie_dict = {
            'id': result['id'],
            'title': result['title'],
            'overview': result['overview']
        }
        movies.append(movie_dict)
    return pd.DataFrame(movies)

def get_movie_details(movie_id):
    movie = tmdb.Movies(movie_id)
    details = movie.info()
    base_image_url = "https://image.tmdb.org/t/p/w500"  # Using 'w500' as an example size
    poster_url = f"{base_image_url}{details['poster_path']}"
    return {
        'id': details['id'],
        'title': details['title'],
        'genres': [genre['name'] for genre in details['genres']],
        'overview': details['overview'],
        'average_rating': details['vote_average'],
        'vote_count': details['vote_count'],
        'popularity': details['popularity'],
        'poster': poster_url
    }

In [43]:
def get_recommendations2(movie_id, num_recommendations=3):
    all_movies_df = search_movies(movie_id)  # DataFrame of all movies
    target_movie_details = get_movie_details(movie_id)  # Details of the target movie

    # Ensure target movie details are included in the movies DataFrame for similarity comparison
    if all_movies_df[all_movies_df['id'] == movie_id].empty:
        all_movies_df = pd.concat([all_movies_df, pd.DataFrame([target_movie_details])], ignore_index=True)

    # Process textual data with TF-IDF
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(all_movies_df['overview'].fillna(''))

    # Compute similarity with linear_kernel
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Get pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[all_movies_df.index[all_movies_df['id'] == movie_id].tolist()[0]]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the most similar movies
    sim_scores = sim_scores[1:num_recommendations+1]  

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top most similar movies
    return all_movies_df.iloc[movie_indices].reset_index(drop=True)

In [47]:
get_recommendations2(345)

id                title   
0  754307  Lisboa-Boa 345 A.E.  \
1  798742              3:45 PM   
2  793443         Roll (3,4,5)   

                                            overview genres  average_rating   
0  On All Saint’s Day 1755, around 09:40 hours, L...    NaN             NaN  \
1  A girl sinks into contemplation during a picni...    NaN             NaN   
2  1.If you could go to space right now would you...    NaN             NaN   

   vote_count  popularity poster  
0         NaN         NaN    NaN  
1         NaN         NaN    NaN  
2         NaN         NaN    NaN

In [51]:
import requests
from bs4 import BeautifulSoup

def scrape_imdb_movie_details(movie_id):
    url = f"https://www.imdb.com/title/{movie_id}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
 # Hypothetical example based on possible HTML structure
# Title
    title_tag = soup.find('h1', class_='TitleHeader__TitleText-sc-1wu6n3d-0')
    title = title_tag.text.strip() if title_tag else 'Title Not Found'    

    # Genres (assuming they are now in a list with a specific class)
    genre_tags = soup.select('ul.GenresList__Genres-sc-1234abcd-0 a')
    genres = [genre.text for genre in genre_tags] if genre_tags else ['No Genres Found']

    
    # Attempt to find the poster URL with updated selectors
    poster_tag = soup.find('div', class_='ipc-poster').find('img', recursive=False) if soup.find('div', class_='ipc-poster') else None
    poster_url = poster_tag['src'] if poster_tag else 'No Poster URL'
 
    
    # Overview
    overview_tag = soup.find('span', class_='sc-16ede01-1 kgphFu')  # This class name looks auto-generated and may change
    overview = overview_tag.text.strip() if overview_tag else 'No Overview Found'
    
    return {
        'title': title,
        'poster_url': poster_url,
        'genres': genres,
        'overview': overview
    }

# Example usage
movie_details = scrape_imdb_movie_details('tt0133093')  # The Matrix
print(movie_details)


{'title': 'Title Not Found', 'poster_url': 'No Poster URL', 'genres': ['No Genres Found'], 'overview': 'No Overview Found'}


In [52]:
import requests

def get_movie_details_from_tmdb(movie_id):
    api_key = 'b7cd3340a794e5a2f35e3abb820b497f'
    url = f'https://api.themoviedb.org/3/movie/{movie_id}'
    params = {'api_key': api_key}
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return {
            'title': data.get('title', 'Title Not Found'),
            'poster_url': f"https://image.tmdb.org/t/p/w500{data.get('poster_path', '')}" if data.get('poster_path') else 'No Poster URL',
            'genres': [genre['name'] for genre in data.get('genres', [])],
            'overview': data.get('overview', 'No Overview Found')
        }
    else:
        return 'Failed to fetch data'

# Example usage
movie_details = get_movie_details_from_tmdb(550)  # Replace 550 with the TMDb movie ID you're interested in
print(movie_details)


{'title': 'Fight Club', 'poster_url': 'https://image.tmdb.org/t/p/w500/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg', 'genres': ['Drama'], 'overview': 'A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground "fight clubs" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.'}


In [66]:
import requests

def get_movie_details(movie_id):
    api_key = 'b7cd3340a794e5a2f35e3abb820b497f'
    movie_url = f'https://api.themoviedb.org/3/movie/{movie_id}'
    credits_url = f'https://api.themoviedb.org/3/movie/{movie_id}/credits'
    params = {'api_key': api_key}
    
    movie_response = requests.get(movie_url, params=params)
    credits_response = requests.get(credits_url, params=params)
    
    if movie_response.status_code == 200 and credits_response.status_code == 200:
        movie_data = movie_response.json()
        credits_data = credits_response.json()
        
        # Director(s)
        directors = [crew['name'] for crew in credits_data.get('crew', []) if crew['job'] == 'Director']
        
        # Top 3 actors
        actors = [cast['name'] for cast in credits_data.get('cast', [])[:3]]
        
        return {
            'id':movie_data.get(movie_id) ,
            'title': movie_data.get('title', 'Title Not Found'),
            'poster_url': f"https://image.tmdb.org/t/p/w500{movie_data.get('poster_path', '')}" if movie_data.get('poster_path') else 'No Poster URL',
            'genres': [genre['name'] for genre in movie_data.get('genres', [])],
            'overview': movie_data.get('overview', 'No Overview Found'),
            'directors': directors,
            'actors': actors,
            'vote_average': movie_data.get('vote_average', 'Rating Not Available'),
            'vote_count': movie_data.get('vote_count', 'Votes Not Available')
        }
    else:
        return 'Failed to fetch data'

# Example usage
movie_details = get_movie_details_from_tmdb(550)  # Replace 550 with the TMDb movie ID you're interested in

movie_details

{'title': 'Fight Club',
 'poster_url': 'https://image.tmdb.org/t/p/w500/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg',
 'genres': ['Drama'],
 'overview': 'A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground "fight clubs" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.',
 'directors': ['David Fincher'],
 'actors': ['Edward Norton', 'Brad Pitt', 'Helena Bonham Carter'],
 'vote_average': 8.44,
 'vote_count': 28109}

In [81]:
movie_id = 123  # Replace with the actual movie ID you want to fetch

api_key = 'b7cd3340a794e5a2f35e3abb820b497f'
movie_url = f'https://api.themoviedb.org/3/movie/{movie_id}'
credits_url = f'https://api.themoviedb.org/3/movie/{movie_id}/credits'
params = {'api_key': api_key}
    

movie_response = requests.get(movie_url, params=params)
print(movie_response.json())

# Use movie_id instead of id here
movie_details = get_movie_details(movie_id)





{'adult': False, 'backdrop_path': '/x9McE1WFKnAHludiY7xfd7modDC.jpg', 'belongs_to_collection': None, 'budget': 4000000, 'genres': [{'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 14, 'name': 'Fantasy'}], 'homepage': '', 'id': 123, 'imdb_id': 'tt0077869', 'original_language': 'en', 'original_title': 'The Lord of the Rings', 'overview': "The Fellowship of the Ring embark on a journey to destroy the One Ring and end Sauron's reign over Middle-earth.", 'popularity': 31.564, 'poster_path': '/liW0mjvTyLs7UCumaHhx3PpU4VT.jpg', 'production_companies': [{'id': 286, 'logo_path': None, 'name': 'Fantasy Films', 'origin_country': ''}, {'id': 4921, 'logo_path': None, 'name': 'Bakshi Productions', 'origin_country': ''}, {'id': 141322, 'logo_path': None, 'name': 'Saul Zaentz Film Productions', 'origin_country': ''}, {'id': 60, 'logo_path': '/1SEj4nyG3JPBSKBbFhtdcHRaIF9.png', 'name': 'United Artists', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'ES', 'name

AttributeError: 'dict' object has no attribute 'id'